# Opinion Detection - Unsupervised

In [ ]:
import pandas as pd
import numpy as np
import text_normalizer as tn
import nltk
nltk.download('movie_reviews')
import warnings
warnings.filterwarnings('ignore')

### <font color='orange'>Load and Preprocess Data</font>

In [ ]:
from nltk.corpus import movie_reviews as mr
from random import shuffle, seed
seed(42)

documents = [(mr.raw(file_id), 1 if category == 'pos' else 0) for category in mr.categories() for file_id in mr.fileids(category)]
shuffle(documents)

In [ ]:
df_train, df_test = documents[:1600], documents[1600:]
X_train = [item[0] for item in df_train] 
#no supervisat --> no etiquetes

X_test = [item[0] for item in df_test]
y_test = [item[1] for item in df_test]

X_train_cleaned = tn.normalize_corpus(X_train)
X_test_cleaned = tn.normalize_corpus(X_test)